In [ ]:

import tensorflow as tf
import numpy as np
import numpy.typing as npt

# Download model from hub

In [ ]:
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/speech_embedding/1')
tf.saved_model.save(hub_model, "../models/conv_speech_embedding", signatures={x: hub_model.signatures[x] for x in dict(hub_model.signatures).keys()})

## or load from already downloaded

In [ ]:
# @title: Load and Save TensorFlow Hub Model
hub_model = tf.saved_model.load("../models/conv_speech_embedding")

## convert to tflite
Można podlądać w https://netron.app/

In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([hub_model.signatures["default"]])
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # TFLite native ops
    tf.lite.OpsSet.SELECT_TF_OPS     # Include TF ops in TFLite
]
tflite_model = converter.convert()
with open("../models/conv_speech_embedding.tflite", "wb") as f:
    f.write(tflite_model)

## Load models

In [ ]:
def get_model1():
    model1 = tf.lite.Interpreter(model_path=str('../models/embedding_model.tflite'), experimental_preserve_all_tensors=True)
    model1.resize_tensor_input(
        model1.get_input_details()[0]['index'],
        [1, 76, 32, 1],
        strict=True)
    model1.allocate_tensors()
    model1_in = model1.get_input_details()[0]['index']
    model1_out = model1.get_output_details()[0]['index']
    return model1, model1_in, model1_out

def get_model2(samples):
    model2 = tf.lite.Interpreter(model_path=str('../models/conv_speech_embedding.tflite'), experimental_preserve_all_tensors=True)
    model2.resize_tensor_input(
        model2.get_input_details()[0]['index'],
        [1, samples],
        strict=True)
    model2.allocate_tensors()
    model2_in = model2.get_input_details()[0]['index']
    model2_out = model2.get_output_details()[0]['index']
    return model2, model2_in, model2_out


## Compare original model from HUB with converted tflite

In [ ]:
data = np.random.rand(12480).astype(np.float32)
res1 = hub_model.signatures['default'](default=data.reshape(1, -1))['default']
model2, model2_in, model2_out = get_model2(data.shape[0])
model2.set_tensor(model2_in, data.reshape(1, -1))
model2.invoke()
res2 = model2.get_tensor(model2_out)
diff = np.abs(res2 - res1)
print("Difference between outputs absolute:", np.max(diff), np.mean(diff), np.min(diff))
diff = np.abs(res2 - res1) / np.maximum(np.abs(res2), np.abs(res1))
print("Difference between outputs relative:", np.max(diff), np.mean(diff), np.min(diff))


## Get mel spec from random

In [ ]:
data = np.random.rand(12480).astype(np.float32)
model2, model2_in, model2_out = get_model2(data.shape[0])
model2.set_tensor(model2_in, data.reshape(1, -1))
model2.invoke()
output2 = model2.get_tensor(model2_out)
print(output2.shape)
mel_output = model2.get_tensor(67)
print(mel_output.shape)

model1, model1_in, model1_out = get_model1()
model1.set_tensor(model1_in, mel_output)
model1.invoke()
output1 = model1.get_tensor(model1_out)
print(output1.shape)

diff = np.abs(model1.get_tensor(43) - model2.get_tensor(68))
print("Difference between outputs:", np.max(diff), np.mean(diff), np.min(diff))


In [ ]:
data = np.random.rand(12480).astype(np.float32)

def check_layers(input_index, output_index, callbacks: list[any]):
    model2, model2_in, model2_out = get_model2(data.shape[0])
    model2.set_tensor(model2_in, data.reshape(1, -1))
    model2.invoke()
    x = model2.get_tensor(input_index)
    for callback in callbacks:
        x = callback(x)
    exp = model2.get_tensor(output_index)
    if input_index == output_index:
        diff = np.abs(x)
    else:
        diff = np.abs(exp - x)
    print("Difference between outputs:", np.max(diff), np.mean(diff), np.min(diff))

def layer_pad(x: 'npt.NDArray[np.float32]') -> 'npt.NDArray[np.float32]':
    return np.pad(x, ((0, 0), (0, 0), (1, 1), (0, 0)), mode='constant', constant_values=0)

def layer_conv1(x: 'npt.NDArray[np.float32]') -> 'npt.NDArray[np.float32]':
    conv1_filter = np.load('../models/vars/conv1_filter.npy')
    conv1_mul = np.load('../models/vars/conv1_mul.npy')
    conv1_filter = conv1_filter * conv1_mul[:, np.newaxis, np.newaxis, np.newaxis]
    conv1_filter = np.transpose(conv1_filter, (1, 2, 3, 0))
    x = tf.nn.conv2d(x, filters=conv1_filter, strides=[1, 1, 1, 1], padding='VALID', data_format='NHWC').numpy()
    x = np.maximum(0, x)
    conv1_add = np.load('../models/vars/conv1_add.npy')
    return x + conv1_add

def layer_relu(x: 'npt.NDArray[np.float32]') -> 'npt.NDArray[np.float32]':
    x = np.maximum(0, x) + 0.2 * (x - np.maximum(0, x))
    return np.maximum(-0.4, x)

def layer_conv(filter_file: str, bias_file: str, padding: str):
    filter = np.load('../models/vars/' + filter_file)
    filter = np.transpose(filter, (1, 2, 3, 0))
    bias = np.load('../models/vars/' + bias_file)
    def exec(x: 'npt.NDArray[np.float32]') -> 'npt.NDArray[np.float32]':
        x = tf.nn.conv2d(x, filters=filter, strides=[1, 1, 1, 1], padding=padding, data_format='NHWC').numpy()
        x = x + bias
        return x
    return exec

def max_pool2x2(x: 'npt.NDArray[np.float32]') -> 'npt.NDArray[np.float32]':
    return tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID', data_format='NHWC').numpy()

def max_pool1x2(x: 'npt.NDArray[np.float32]') -> 'npt.NDArray[np.float32]':
    return tf.nn.max_pool2d(x, ksize=[1, 1, 2, 1], strides=[1, 1, 2, 1], padding='VALID', data_format='NHWC').numpy()


check_layers(67, 67, [])
check_layers(67, 68, [layer_pad])
check_layers(67, 71, [layer_pad, layer_conv1])
check_layers(67, 73, [layer_pad, layer_conv1, layer_relu])
check_layers(67, 74, [layer_pad, layer_conv1, layer_relu, layer_conv('conv2_filter.npy', 'conv2_bias.npy', 'SAME')])
check_layers(67, 76, [layer_pad, layer_conv1, layer_relu, layer_conv('conv2_filter.npy', 'conv2_bias.npy', 'SAME'), layer_relu])
check_layers(76, 77, [layer_conv('conv3_filter.npy', 'conv3_bias.npy', 'VALID')])
check_layers(76, 79, [layer_conv('conv3_filter.npy', 'conv3_bias.npy', 'VALID'), layer_relu])
check_layers(76, 80, [layer_conv('conv3_filter.npy', 'conv3_bias.npy', 'VALID'), layer_relu, max_pool2x2])
check_layers(80, 81, [layer_conv('conv4_filter.npy', 'conv4_bias.npy', 'SAME')])
check_layers(80, 83, [layer_conv('conv4_filter.npy', 'conv4_bias.npy', 'SAME'), layer_relu])
check_layers(83, 84, [layer_conv('conv5_filter.npy', 'conv5_bias.npy', 'VALID')])
check_layers(83, 86, [layer_conv('conv5_filter.npy', 'conv5_bias.npy', 'VALID'), layer_relu])
check_layers(86, 87, [layer_conv('conv6_filter.npy', 'conv6_bias.npy', 'SAME')])
check_layers(86, 89, [layer_conv('conv6_filter.npy', 'conv6_bias.npy', 'SAME'), layer_relu])
check_layers(89, 90, [layer_conv('conv7_filter.npy', 'conv7_bias.npy', 'VALID')])
check_layers(89, 92, [layer_conv('conv7_filter.npy', 'conv7_bias.npy', 'VALID'), layer_relu])
check_layers(89, 93, [layer_conv('conv7_filter.npy', 'conv7_bias.npy', 'VALID'), layer_relu, max_pool1x2])


check_layers(67, 93, [layer_pad, layer_conv1, layer_relu, layer_conv('conv2_filter.npy', 'conv2_bias.npy', 'SAME'), layer_relu,
    layer_conv('conv3_filter.npy', 'conv3_bias.npy', 'VALID'), layer_relu, max_pool2x2,
    layer_conv('conv4_filter.npy', 'conv4_bias.npy', 'SAME'), layer_relu,
    layer_conv('conv5_filter.npy', 'conv5_bias.npy', 'VALID'), layer_relu,
    layer_conv('conv6_filter.npy', 'conv6_bias.npy', 'SAME'), layer_relu,
    layer_conv('conv7_filter.npy', 'conv7_bias.npy', 'VALID'), layer_relu, max_pool1x2,
])



In [ ]:
from pprint import pprint

model2, model2_in, model2_out = get_model2(data.shape[0])

print(dir(model2))
print(model2.get_input_details())
print(model2.get_output_details())
pprint(model2.get_tensor_details())
